In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import networkx as nx
import matplotlib.pyplot as plt
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/iupui-link-prediction/features.csv
/kaggle/input/iupui-link-prediction/sample_submission.csv
/kaggle/input/iupui-link-prediction/train_edges.csv


In [2]:
train_file = "/kaggle/input/iupui-link-prediction/train_edges.csv";
sample_sub = "/kaggle/input/iupui-link-prediction/sample_submission.csv"
test_file = "/kaggle/input/iupui-link-prediction/features.csv";

In [3]:
def calc_index(G):
    
    s = sorted(G.nodes);
    num_nodes = max(s)+1;
    
    #Store it into a matrix for self preservation
    #num_nodes = G.number_of_nodes();
    common_neigh = np.zeros((num_nodes,num_nodes));
    salton = common_neigh.copy();
    jaccard = common_neigh.copy();
    sorensen = common_neigh.copy();
    hub_promoted = common_neigh.copy();
    hub_depressed = common_neigh.copy();
    
    for i in s:
        if (i % 100 == 0):
            print(i)
        for j in s:
            if (i != j):
                deg_i = G.degree[i];
                deg_j = G.degree[j];
                min_deg = deg_i;
                max_deg = deg_j;
                if (deg_j < min_deg):
                    min_deg = deg_j;
                    max_deg = deg_i;

                num_com = len(sorted(nx.common_neighbors(G,i,j)));
                common_neigh[i,j] = num_com;

                if (num_com > 0):
                    salt = num_com / math.sqrt(deg_i * deg_j);
                    jac = num_com / (deg_i + deg_j - num_com);
                    sor = 2 * num_com / (deg_i + deg_j);
                    hub_d = num_com / min_deg;
                    hub_p = num_com / max_deg;

                    salton[i,j] = salt;
                    jaccard[i,j] = jac;
                    sorensen[i,j] = sor;
                    hub_promoted[i,j] = hub_p;
                    hub_depressed[i,j] = hub_d;
                
    
    return(common_neigh,salton,jaccard,sorensen,hub_promoted,hub_depressed);

def calc_common_neigh(G):
    #Store it into a matrix for self preservation
    num_nodes = G.number_of_nodes();
    common_neigh = np.zeros((num_nodes,num_nodes));
    
    for i in range(num_nodes):
        if (i % 100 == 0):
            print(i)
        for j in range(num_nodes):
            if (i != j):
                num_com = len(sorted(nx.common_neighbors(G,i,j)));
                common_neigh[i,j] = num_com
    
    return(common_neigh);

In [4]:
#Load the input files
import csv

G = nx.Graph()

with open(train_file) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1]=='1':
            edge= row[0].split('-')
            G.add_edge(int(edge[0]),int(edge[1]))

In [5]:
#Node info
print(G.number_of_nodes());
#nx.draw(G,node_size = 1);

2506


In [6]:
# next let's read in the possible edges we need to classify from the test file
Gsub = nx.Graph()

with open(sample_sub) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1]=='1' or row[1]=='0':
            edge= row[0].split('-')
            Gsub.add_edge(int(edge[0]),int(edge[1]))

In [7]:
#print(sorted(G.nodes))
s = sorted(G.nodes);
#print(s)
a = 0;
for i in range(len(s)-1):
    if(s[i+1] - s[i] != 1):
        a += 1;
print(a)
#print(G.degree[10])

187


In [8]:
#n = calc_common_neigh(G)
com,salt,jac,sor,hub_p,hub_d = calc_index(G);

#print(com,salt,jac,sor,hub_p,hub_d);

0
100
200
400
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2400
2500
2600
2700


In [9]:
#Now I need to make this into something that I can feed into a neural net like nothing
num_nodes = max(s)+1;
adj = nx.adjacency_matrix(G);
adj = adj.todense();

datum = np.zeros((num_nodes**2,7));

for i in s:
    if (i % 100 == 0):
            print(i)
    for j in s:
        if (j > i):
            ind = i*num_nodes+j;
            datum[ind,0] = com[i,j];
            datum[ind,1] = salt[i,j];
            datum[ind,2] = jac[i,j];
            datum[ind,3] = sor[i,j];
            datum[ind,4] = hub_p[i,j];
            datum[ind,5] = hub_d[i,j];
            ind_i = s.index(i);
            ind_j = s.index(j);
            
            datum[ind,6] = adj[ind_i,ind_j];
        
#remove the all zero rows
#datum = datum[~np.all(datum == 0, axis=1)];
#Append an all zero row
#oof = np.zeros((1,7));
#datum = np.append(datum,oof);
datum = datum.reshape(-1,7);
print(datum)

0
100
200
400
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2400
2500
2600
2700
[[0.         0.         0.         ... 0.         0.         0.        ]
 [2.         0.10166571 0.01538462 ... 0.01550388 0.66666667 1.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [10]:
#datum = datum.reshape(-1,7)
#print(datum)
#Output the data to a csv for further analysis in Frickin R
#Because I have to for the fricking HW assignment.
df = pd.DataFrame(datum);
df.to_csv("/kaggle/working/train_r.csv",index=False)

In [11]:
#print(G)
#list(G.nodes)         
#print(Gsub.nodes)
# we need to add the nodes from the test data to training network (some are new nodes)
G.add_nodes_from(Gsub.nodes)
print(G.number_of_nodes());

2708


In [12]:
# here is a simple algorithm to get you started...for each possible edge i-j in the test set
# we will count the number of nodes in the training graph they have in common...if they have
# one or more "mutual friends" then we will connect them (this will score about .67 acc on kaggle)
with open('/kaggle/working/stupid_train.csv', 'w') as csvfile:
    fieldnames = ['edge', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    with open(sample_sub) as csvfile2:
        reader = csv.reader(csvfile2, delimiter=',')
        for row in reader:
            if row[1]=='1' or row[1]=='0':
                edge= row[0].split('-')

                i=int(edge[0])
                j=int(edge[1])
                
                # here networkx has a common_neighbors function
                num_com_neigh=len(sorted(nx.common_neighbors(G, i, j)))
                y=0
                #if num_com_neigh>0:
                #    y=1

                edg_out=str(i)+"-"+str(j)    
                writer.writerow({'edge': edg_out, 'label': y})
                #print(edg_out, num_com_neigh, y)

In [13]:
#Output the wanted stuff
#Recalculate with the new stuff
com,salt,jac,sor,hub_p,hub_d = calc_index(G);

out_df = pd.read_csv(sample_sub);
out_df = out_df.values;
sz = np.shape(out_df);

out_file = np.zeros((sz[0],6));
for i in range(sz[0]):
    edge= out_df[i,0];
    edge = edge.split('-');
    j = int(edge[0]);
    k = int(edge[1]);
    #print(j,k)
    
    out_file[i,0] = com[j,k];
    out_file[i,1] = salt[j,k];
    out_file[i,2] = jac[j,k];
    out_file[i,3] = sor[j,k];
    out_file[i,4] = hub_p[j,k];
    out_file[i,5] = hub_d[j,k];

print(out_file)
#print(np.mean(out_file))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [14]:
#output the outfile
out_file_df = pd.DataFrame(out_file);
out_file_df.to_csv("/kaggle/working/test_r.csv",index=False)